<a href="https://colab.research.google.com/github/Wittgenbot/fine-tuning/blob/main/LLM_as_a_judge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set Up

## Imports

In [ ]:
import ast
import json
import requests
from google.colab import userdata, drive

## Google Drive

In [ ]:
drive.mount('/content/drive')
data_dir = '/content/drive/My Drive/LLM as a Judge/'

Mounted at /content/drive


## Mixtral API Set Up

In [ ]:
MIXTRAL_API_KEY = userdata.get('MIXTRAL_API_KEY')

In [ ]:
def query_mixtral(system_prompt, user_prompt):

    url = "https://api.mistral.ai/v1/chat/completions"

    payload = {
        "model": "open-mixtral-8x7b",
        "stop": ["</s>"],
        "stream": False,
        "messages": [
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": user_prompt
            }
        ]
    }

    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "Authorization": f"Bearer {MIXTRAL_API_KEY}"
    }

    max_attempts = 3
    attempt = 0

    while attempt < max_attempts:
        try:
            response = requests.post(url, json=payload, headers=headers)
            response.raise_for_status()
            data = response.json()
            usage = data.get("usage", {})
            answer = data.get("choices", [{}])[0].get("message", {}).get("content", "")
            break
        except requests.RequestException as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            attempt += 1
            if attempt < max_attempts:
                sleep_time = 2 ** attempt
                print(f"Retrying in {sleep_time} seconds...")
                sleep(sleep_time)
            else:
                print("Maximum retry attempts reached, failing.")

    return answer

# Test Set Up

## Test Questions

In [ ]:
TLP_test_questions = [
    'How does Wittgenstein define the concept of a "proposition" in the Tractatus?',
    'What role does logic play in the structure of reality as presented in the Tractatus?',
    'Can you explain the picture theory of language proposed in the Tractatus?',
    'How does Wittgenstein distinguish between what can be said and what can only be shown?',
    'How are facts and states of affairs conceptualized in the Tractatus?',
    'What is the significance of the limit of language in the Tractatus?',
    'How does the Tractatus address the relationship between language and reality?',
    'What is the purpose of the ladder metaphor in the conclusion of the Tractatus?',
    'What is meant by "propositional logic" in the Tractatus, and how is it significant to the work’s overall argument?',
    'How does the Tractatus critique the possibility of metaphysical propositions?',
    'What is the role of silence in Wittgenstein’s philosophy as expressed in the Tractatus?',
    'What did Wittgenstein mean by "philosophy is not a body of doctrine but an activity" in the Tractatus?',
    'How does Wittgenstein treat the problem of solipsism in the Tractatus?',
    'What does Wittgenstein mean by "The limits of my language mean the limits of my world"?',
    'How does the Tractatus conceptualize the idea of sense and nonsense in language?',
]

PI_test_questions = [
    'In Wittgenstein\'s Philosophical Investigations, what determines the meaning of a word?',
    'How did Wittgenstein use the example of "games" in Philosophical Investigations to illustrate the family resemblance concept?',
    'In Philosophical Investigations, what is the concept of family resemblance?',
    'In the context of Philosophical Investigations, is the existence of a private language possible?',
    'How can we confirm that someone is following a rule, according to Philosophical Investigations?',
    'In Philosophical Investigations, how can the concept of sameness be used to teach a rule?',
    'What is the role of language-games in Wittgenstein\'s Philosophical Investigations?',
    'How is the idea that mental processes form the basis of our understanding of language critiqued in Philosophical Investigations?',
    'Do private mental objects exist according to Wittgenstein’s Philosophical Investigations?',
    'What is the relationship between forms of life and language in Philosophical Investigations?',
    'In Philosophical Investigations, how is it explained that the meaning of a word is its use in language?',
    'How do philosophical problems arise from misunderstandings of language, as discussed in Philosophical Investigations?',
    'What is problematic about the Augustinian view on meaning in Philosophical Investigations?',
    'In Philosophical Investigations, what is said about the misguided nature of philosophical questions?',
    'What is the purpose of the analogy with the toolbox in Philosophical Investigations?',
    'In what ways does the notion of "language-games" in Philosophical Investigations challenge traditional epistemology?',
    'What implications does the critique of private language in Philosophical Investigations have for theories of consciousness?',
    'How is the concept of pain used in Philosophical Investigations to argue against the possibility of a private language?',
    'In what ways does the examination of pain\'s sameness in Philosophical Investigations challenge the understanding of subjective experiences?',
    'What are the limitations of language, as discussed in Wittgenstein’s Philosophical Investigations?',
    'In Philosophical Investigations, what is the relationship between forms of life and understanding philosophy?'
]

test_questions = {
    'TLP': TLP_test_questions,
    'PI': PI_test_questions
}

## Retrieve Wittgenbot "To-Be-Judged" Answers From Google Drive

In [ ]:
wittgenbot_answers_file_path = data_dir + 'wittgenbot_answers.txt'

with open(wittgenbot_answers_file_path, 'r') as file:
    content = file.read()
    wittgenbot_answers = ast.literal_eval(content)

print(wittgenbot_answers)

{'TLP': {'How does Wittgenstein define the concept of a "proposition" in the Tractatus?': 'Wittgenstein defines a "proposition" in the Tractatus Logico-Philosophicus (TLP) as a combination of simple signs that represent the possible states of affairs in the world. A proposition, according to Wittgenstein, is a factual statement that can be true or false. It is a picture of reality, and its structure mirrors the structure of the state of affairs it represents.\n\nWittgenstein\'s concept of a proposition is deeply connected to his theory of language and logic. He argues that the world is composed of facts, and these facts can be represented by propositions. A proposition is a logical picture of a fact, and its truth or falsity depends on whether the elements of the proposition correspond to the elements of the fact.\n\nIn TLP, Wittgenstein introduces the idea of atomic propositions, which are the simplest form of propositions. An atomic proposition is a proposition that cannot be further

## Generate Reference "Golden" Answers Through Mixtral 8x7b

In [ ]:
system_prompt = '''
You are a philosophy expert specialized in the philosophy of Ludwig Wittgenstein. Answer the user's question. Ensure your answer is accurate, thorough, coherent and relevant. Use a tone that is conversational and clear.
'''

user_prompt = '''
Answer this question about Ludwig Wittgenstein's philosophy:
{question}
'''

In [ ]:
golden_answers = {}

for work, questions in test_questions.items():

  golden_answers[work] = {}

  for question in questions:

    formatted_user_prompt = user_prompt.format(question=question)

    answer = query_mixtral(system_prompt, formatted_user_prompt)

    golden_answers[work][question] = answer

print(golden_answers)

{'TLP': {'How does Wittgenstein define the concept of a "proposition" in the Tractatus?': 'hello', 'What role does logic play in the structure of reality as presented in the Tractatus?': 'hello', 'Can you explain the picture theory of language proposed in the Tractatus?': 'hello', 'How does Wittgenstein distinguish between what can be said and what can only be shown?': 'hello', 'How are facts and states of affairs conceptualized in the Tractatus?': 'hello', 'What is the significance of the limit of language in the Tractatus?': 'hello', 'How does the Tractatus address the relationship between language and reality?': 'hello', 'What is the purpose of the ladder metaphor in the conclusion of the Tractatus?': 'hello', 'What is meant by "propositional logic" in the Tractatus, and how is it significant to the work’s overall argument?': 'hello', 'How does the Tractatus critique the possibility of metaphysical propositions?': 'hello', 'What is the role of silence in Wittgenstein’s philosophy as

### Save Results to Google Drive

In [ ]:
golden_answers_file_name = 'golden_answers.txt'
golden_answers_file_path = data_dir + golden_answers_file_name

golden_answers_json = json.dumps(golden_answers, indent=4)

with open(golden_answers_file_path, 'w') as file:
    file.write(golden_answers_json)

print(f'File saved as {golden_answers_file_name}')

{'TLP': {'How does Wittgenstein define the concept of a "proposition" in the Tractatus?': 'hello', 'What role does logic play in the structure of reality as presented in the Tractatus?': 'hello', 'Can you explain the picture theory of language proposed in the Tractatus?': 'hello', 'How does Wittgenstein distinguish between what can be said and what can only be shown?': 'hello', 'How are facts and states of affairs conceptualized in the Tractatus?': 'hello', 'What is the significance of the limit of language in the Tractatus?': 'hello', 'How does the Tractatus address the relationship between language and reality?': 'hello', 'What is the purpose of the ladder metaphor in the conclusion of the Tractatus?': 'hello', 'What is meant by "propositional logic" in the Tractatus, and how is it significant to the work’s overall argument?': 'hello', 'How does the Tractatus critique the possibility of metaphysical propositions?': 'hello', 'What is the role of silence in Wittgenstein’s philosophy as